In [1]:
import numpy as np  # linear algebra
import pandas as pd  # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn import datasets, linear_model
from datetime import datetime
import gc
%matplotlib inline
from IPython.display import display, HTML
from pprint import pprint

## Usage!!!
- Always change maxVal if iterative feature is added..5, for np.nan's add 4, then 3.. maxVal calculates according to this..

In [2]:
class ConfigElements:
    def __init__(self, lag, nameAndGroups, targetVariable="", deleteColumns = True):
        self.lag = lag
        self.nameAndGroups = nameAndGroups
        #If there is target variable, then 5 4 3 2 1, fill the np.nans..Else hold them all in Dataframe..
        self.targetVariable = targetVariable
        self.deleteColumns = deleteColumns

In [3]:
configLag0Target1 = ConfigElements(0,[ ("A",["Agencia_ID"]), ("Ca",["Canal_ID"]), ("R",["Ruta_SAK"]), 
    ("Cl",["Cliente_ID"]),("P",["Producto_ID"]),("ClP",["Cliente_ID","Producto_ID"])   ], "lag0par1", False)
print  configLag0Target1.__dict__

{'nameAndGroups': [('A', ['Agencia_ID']), ('Ca', ['Canal_ID']), ('R', ['Ruta_SAK']), ('Cl', ['Cliente_ID']), ('P', ['Producto_ID']), ('ClP', ['Cliente_ID', 'Producto_ID'])], 'lag': 0, 'targetVariable': 'lag0par1', 'deleteColumns': False}


In [4]:
configLag0Target0 = ConfigElements(0,[ ("A",["Agencia_ID"]), ("Ca",["Canal_ID"]), ("R",["Ruta_SAK"]), 
    ("Cl",["Cliente_ID"]),("P",["Producto_ID"]),("ClP",["Cliente_ID","Producto_ID"])   ])
print  configLag0Target0.__dict__

{'nameAndGroups': [('A', ['Agencia_ID']), ('Ca', ['Canal_ID']), ('R', ['Ruta_SAK']), ('Cl', ['Cliente_ID']), ('P', ['Producto_ID']), ('ClP', ['Cliente_ID', 'Producto_ID'])], 'lag': 0, 'targetVariable': '', 'deleteColumns': True}


In [5]:
configLag2Target1 = ConfigElements(2,[ ("lag2A",["Semana","Agencia_ID"]), ("lag2Ca",["Semana","Canal_ID"]),
    ("lag2R",["Semana","Ruta_SAK"]), ("lag2Cl",["Semana","Cliente_ID"]),("lag2P",["Semana","Producto_ID"]), 
    ("lag2S",["Semana"])   ], "Lag2Target2", False)
print  configLag2Target1.__dict__

{'nameAndGroups': [('lag2A', ['Semana', 'Agencia_ID']), ('lag2Ca', ['Semana', 'Canal_ID']), ('lag2R', ['Semana', 'Ruta_SAK']), ('lag2Cl', ['Semana', 'Cliente_ID']), ('lag2P', ['Semana', 'Producto_ID']), ('lag2S', ['Semana'])], 'lag': 2, 'targetVariable': 'Lag2Target2', 'deleteColumns': False}


In [6]:
configLag2Target0 = ConfigElements(2,[ ("lag2A",["Semana","Agencia_ID"]), ("lag2Ca",["Semana","Canal_ID"]),
    ("lag2R",["Semana","Ruta_SAK"]), ("lag2Cl",["Semana","Cliente_ID"]),("lag2P",["Semana","Producto_ID"]), 
    ("lag2S",["Semana"])   ])
print  configLag2Target1.__dict__

{'nameAndGroups': [('lag2A', ['Semana', 'Agencia_ID']), ('lag2Ca', ['Semana', 'Canal_ID']), ('lag2R', ['Semana', 'Ruta_SAK']), ('lag2Cl', ['Semana', 'Cliente_ID']), ('lag2P', ['Semana', 'Producto_ID']), ('lag2S', ['Semana'])], 'lag': 2, 'targetVariable': 'Lag2Target2', 'deleteColumns': False}


##  Test for filling missing values that are already on a column, no need to create a target variable..

In [7]:
#configWithoutLag2 = ConfigElements(0,[ ("A",["Agencia_ID"]), ("Ca",["Canal_ID"]), ("R",["Ruta_SAK"]), 
#    ("Cl",["Cliente_ID"]),("P",["Producto_ID"]),("ClP",["Cliente_ID","Producto_ID"])   ], "lag2par1")
#print  configWithLag.__dict__

In [8]:
class FeatureEngineering:
    #ValidationStart = 0
    #lag = 0
    #trainHdfPath = ""
    #trainHdfFile = ""
    #testHdfPath = ""
    #testHdfFile = ""
    #train = pd.DataFrame()
    #test = pd.DataFrame()
    
    def __init__(self, ValidationStart, ValidationSmallerThan, trainHdfPath, trainHdfFile, testHdfPath, testHdfFile, 
                 testTypes, trainTypes, trainCsvPath, testCsvPath, maxLag=0):
        self.ValidationStart = ValidationStart
        self.ValidationSmallerThan = ValidationSmallerThan
        self.maxLag = maxLag
        self.trainHdfPath = trainHdfPath
        self.trainHdfFile = trainHdfFile
        self.testHdfPath = testHdfPath
        self.testHdfFile = testHdfFile
        self.testTypes = testTypes
        self.trainTypes = trainTypes
        self.trainCsvPath = trainCsvPath
        self.testCsvPath = testCsvPath
        
    @staticmethod
    def __printDataFrameBasics__(data):
        display(data.head(2))
        print data.dtypes
        gc.collect()
        print(data.info(memory_usage=True))
        
    @staticmethod    
    def __changeIndexTypeToLowerMemory(data):
        ##########
        #This is very critical, i accept max number is 2^32. Also, if don't do that, memory gets so much higher..
        ##########
        data.index = data.index.astype('uint32')
        
    def ReadHdf(self, trainOrTestOrBoth):
        '''Reads and holds Df in object memory'''
        if trainOrTestOrBoth == '':
            raise "qweqweq"
            
        if trainOrTestOrBoth == 'train' or trainOrTestOrBoth=='both':
            self.train = pd.read_hdf(self.trainHdfPath,self.trainHdfFile)
            FeatureEngineering.__changeIndexTypeToLowerMemory(self.train)
            FeatureEngineering.__printDataFrameBasics__(self.train)
        if trainOrTestOrBoth == 'test' or trainOrTestOrBoth=='both':
            self.test = pd.read_hdf(self.testHdfPath,self.testHdfFile)
            FeatureEngineering.__changeIndexTypeToLowerMemory(self.test)
            FeatureEngineering.__printDataFrameBasics__(self.test)
        
    def ReadCsv(self, trainOrTestOrBoth):
        '''Reads and holds Df in object memory'''
        if trainOrTestOrBoth == '':
            raise "qweqweq"  
        if trainOrTestOrBoth == 'train' or trainOrTestOrBoth=='both':
            self.train = pd.read_csv(self.trainCsvPath, usecols=self.trainTypes.keys(), dtype=self.trainTypes)
            #__changeIndexTypeToLowerMemory(self.train)
            FeatureEngineering.__printDataFrameBasics__(self.train)
        if trainOrTestOrBoth == 'test' or trainOrTestOrBoth=='both':
            self.test = pd.read_csv(self.testCsvPath, usecols=self.testTypes.keys(), dtype=self.testTypes)
            #__changeIndexTypeToLowerMemory(self.test)
            FeatureEngineering.__printDataFrameBasics__(self.test)

    @staticmethod
    def ConvertCsvToHdf(csvPath, HdfPath, HdfName, ColumnTypeDict ):
        tempDf = pd.read_csv(csvPath, usecols=ColumnTypeDict.keys(), dtype=ColumnTypeDict)
        tempDf.to_hdf(HdfPath, HdfName, format='t')
        print "ConvertCsvToHdf is done.."

    def Preprocess(self, trainOrTestOrBoth, columnFunctionTypeList):
        '''columnFunctionTypeList = [ ['C1',Func1,Type], ['C2',Func2,Type],..    ]'''
        for column, func, localType in columnFunctionTypeList:
            if trainOrTestOrBoth == 'train' or trainOrTestOrBoth=='both':
                self.train.loc[:,column] =  self.train[column].apply(func).astype(localType)
            elif trainOrTestOrBoth == 'test' or trainOrTestOrBoth=='both':
                self.test.loc[:,column] =  self.test[column].apply(func).astype(localType)
            
    def SaveDataFrameToHdf(self,trainOrTestOrBoth):
        if trainOrTestOrBoth == 'train' or trainOrTestOrBoth=='both':
            self.train.to_hdf(self.trainHdfPath, self.trainHdfFile, format='t')
        if trainOrTestOrBoth == 'test' or trainOrTestOrBoth=='both':
            self.test.to_hdf(self.testHdfPath, self.testHdfFile, format='t')
        
    def AddDemandaGeneralMean(self):
        DemandaMeanWithoutLag = self.train.loc[self.train.loc[:,'Semana'] < self.ValidationStart]
        self.train.loc[:,"DemandaGeneralMean"] = DemandaMeanWithoutLag['Demanda_uni_equil'].mean()
        self.train.loc[:,"DemandaGeneralMean"] = self.train.loc[:,"DemandaGeneralMean"].astype('float32')
        display(self.train)
        
    def AddConfigurableFeaturesToTrain(self, config):
        if config.lag > self.maxLag:
            self.maxLag = config.lag
        
        lag = 0 if config.lag == 0 else config.lag-1
        tempData = self.train.loc[self.train.loc[:,'Semana'] < self.ValidationStart - lag]
        #display(tempData)
        if(config.lag != 0):
            tempData.loc[:,'Semana'] = tempData.loc[:,'Semana'].apply(lambda x:x + config.lag)
        #display(tempData)
        
        #Means iterative.. eliminate as long as np.nan exists..If there is already one, don't create, use the existing
        if config.targetVariable != "" and  config.targetVariable not in self.train.columns:
            self.train.loc[:,config.targetVariable] = np.nan
        
        for name,groups in config.nameAndGroups:
            if name not in self.train.columns:
                print "{} is not in columns..".format(name)
                groupedDataframe =  pd.DataFrame({name:tempData.groupby(groups)
                    ['Demanda_uni_equil'].mean().astype('float32')})
                #join..
                
                display(groupedDataframe)
                self.train = self.train.merge( groupedDataframe, left_on=groups,
                                  right_index=True, how='left', sort=False,copy=False)
            else:
                print "{} is in columns..".format(name)
            
            display(self.train)
            
            #Means iterative..!!!!!
            if config.targetVariable != "":
                self.train.loc[pd.isnull(self.train[config.targetVariable]), 
                    config.targetVariable] = self.train.loc[pd.isnull(self.train[config.targetVariable]), name]
                count = self.train.loc[self.train['Semana'] >= self.ValidationStart,config.targetVariable].isnull().sum()
                print "Count of missing numbers after {} in validation part in column {} is {}".format(name, 
                    config.targetVariable,str(count))
                #display(self.train)
                #If column is already in Dataframe and we want to fill target variable, this deletes columns!!!
                if(config.deleteColumns):
                    self.train.drop(name, axis=1, inplace=True)
                gc.collect()
                if count == 0:
                    break
        display(self.train)        
        gc.collect()
        return 
    def DeleteLaggedWeeksFromTrain(self):
        self.train = self.train.loc[self.train.loc[:,'Semana']>= 3 + self.maxLag]
        display(self.train)

In [12]:
FE.AddConfigurableFeaturesToTrain(configLag0Target1)

A is not in columns..


,A
Agencia_ID,
1,5.537601
2,4.320922
3,4.546874
4,11.512936


,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Demanda_uni_equil,lag0par1,A
0,3,1,7,3301,15766,1200,1.386294,NaN,5.537601
1,3,2,7,3301,15766,1200,1.609438,NaN,4.320922
2,3,1,7,3301,15766,1200,3.713572,NaN,5.537601
3,3,2,7,3301,15766,1300,1.098612,NaN,4.320922
4,3,3,7,3301,15900,1300,1.386294,NaN,4.546874
5,3,2,7,3301,15766,1300,6.908755,NaN,4.320922
6,4,3,7,3301,15766,1200,11.512936,NaN,4.546874
7,4,2,7,3301,15766,1300,11.512936,NaN,4.320922
8,4,1,7,3301,15766,1300,11.512936,NaN,5.537601
9,4,4,7,3301,15900,1400,11.512936,NaN,11.512936


Count of missing numbers after A in validation part in column lag0par1 is 4
Ca is not in columns..


,Ca
Canal_ID,
2,2.397895
7,5.372375


...

Count of missing numbers after R in validation part in column lag0par1 is 1
Cl is not in columns..


,Cl
Cliente_ID,
15766,5.176513
15900,5.099042


,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Demanda_uni_equil,lag0par1,A,Ca,R,Cl
0,3,1,7,3301,15766,1200,1.386294,5.537601,5.537601,5.372375,5.159912,5.176513
1,3,2,7,3301,15766,1200,1.609438,4.320922,4.320922,5.372375,5.159912,5.176513
2,3,1,7,3301,15766,1200,3.713572,5.537601,5.537601,5.372375,5.159912,5.176513
3,3,2,7,3301,15766,1300,1.098612,4.320922,4.320922,5.372375,5.159912,5.176513
4,3,3,7,3301,15900,1300,1.386294,4.546874,4.546874,5.372375,5.159912,5.099042
5,3,2,7,3301,15766,1300,6.908755,4.320922,4.320922,5.372375,5.159912,5.176513
6,4,3,7,3301,15766,1200,11.512936,4.546874,4.546874,5.372375,5.159912,5.176513
7,4,2,7,3301,15766,1300,11.512936,4.320922,4.320922,5.372375,5.159912,5.176513
8,4,1,7,3301,15766,1300,11.512936,5.537601,5.537601,5.372375,5.159912,5.176513
9,4,4,7,3301,15900,1400,11.512936,11.512936,11.512936,5.372375,5.159912,5.099042


Count of missing numbers after Cl in validation part in column lag0par1 is 0


,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Demanda_uni_equil,lag0par1,A,Ca,R,Cl
0,3,1,7,3301,15766,1200,1.386294,5.537601,5.537601,5.372375,5.159912,5.176513
1,3,2,7,3301,15766,1200,1.609438,4.320922,4.320922,5.372375,5.159912,5.176513
2,3,1,7,3301,15766,1200,3.713572,5.537601,5.537601,5.372375,5.159912,5.176513
3,3,2,7,3301,15766,1300,1.098612,4.320922,4.320922,5.372375,5.159912,5.176513
4,3,3,7,3301,15900,1300,1.386294,4.546874,4.546874,5.372375,5.159912,5.099042
5,3,2,7,3301,15766,1300,6.908755,4.320922,4.320922,5.372375,5.159912,5.176513
6,4,3,7,3301,15766,1200,11.512936,4.546874,4.546874,5.372375,5.159912,5.176513
7,4,2,7,3301,15766,1300,11.512936,4.320922,4.320922,5.372375,5.159912,5.176513
8,4,1,7,3301,15766,1300,11.512936,5.537601,5.537601,5.372375,5.159912,5.176513
9,4,4,7,3301,15900,1400,11.512936,11.512936,11.512936,5.372375,5.159912,5.099042


In [13]:
FE.AddConfigurableFeaturesToTrain(configLag0Target0)

A is in columns..


,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Demanda_uni_equil,lag0par1,A,Ca,R,Cl
0,3,1,7,3301,15766,1200,1.386294,5.537601,5.537601,5.372375,5.159912,5.176513
1,3,2,7,3301,15766,1200,1.609438,4.320922,4.320922,5.372375,5.159912,5.176513
2,3,1,7,3301,15766,1200,3.713572,5.537601,5.537601,5.372375,5.159912,5.176513
3,3,2,7,3301,15766,1300,1.098612,4.320922,4.320922,5.372375,5.159912,5.176513
4,3,3,7,3301,15900,1300,1.386294,4.546874,4.546874,5.372375,5.159912,5.099042
5,3,2,7,3301,15766,1300,6.908755,4.320922,4.320922,5.372375,5.159912,5.176513
6,4,3,7,3301,15766,1200,11.512936,4.546874,4.546874,5.372375,5.159912,5.176513
7,4,2,7,3301,15766,1300,11.512936,4.320922,4.320922,5.372375,5.159912,5.176513
8,4,1,7,3301,15766,1300,11.512936,5.537601,5.537601,5.372375,5.159912,5.176513
9,4,4,7,3301,15900,1400,11.512936,11.512936,11.512936,5.372375,5.159912,5.099042


Ca is in columns..


,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Demanda_uni_equil,lag0par1,A,Ca,R,Cl
0,3,1,7,3301,15766,1200,1.386294,5.537601,5.537601,5.372375,5.159912,5.176513
1,3,2,7,3301,15766,1200,1.609438,4.320922,4.320922,5.372375,5.159912,5.176513
2,3,1,7,3301,15766,1200,3.713572,5.537601,5.537601,5.372375,5.159912,5.176513
3,3,2,7,3301,15766,1300,1.098612,4.320922,4.320922,5.372375,5.159912,5.176513
4,3,3,7,3301,15900,1300,1.386294,4.546874,4.546874,5.372375,5.159912,5.099042
5,3,2,7,3301,15766,1300,6.908755,4.320922,4.320922,5.372375,5.159912,5.176513
6,4,3,7,3301,15766,1200,11.512936,4.546874,4.546874,5.372375,5.159912,5.176513
7,4,2,7,3301,15766,1300,11.512936,4.320922,4.320922,5.372375,5.159912,5.176513
8,4,1,7,3301,15766,1300,11.512936,5.537601,5.537601,5.372375,5.159912,5.176513
9,4,4,7,3301,15900,1400,11.512936,11.512936,11.512936,5.372375,5.159912,5.099042


R is in columns..


...

,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Demanda_uni_equil,lag0par1,A,Ca,R,Cl,P
0,3,1,7,3301,15766,1200,1.386294,5.537601,5.537601,5.372375,5.159912,5.176513,3.701200
1,3,2,7,3301,15766,1200,1.609438,4.320922,4.320922,5.372375,5.159912,5.176513,3.701200
2,3,1,7,3301,15766,1200,3.713572,5.537601,5.537601,5.372375,5.159912,5.176513,3.701200
3,3,2,7,3301,15766,1300,1.098612,4.320922,4.320922,5.372375,5.159912,5.176513,6.483907
4,3,3,7,3301,15900,1300,1.386294,4.546874,4.546874,5.372375,5.159912,5.099042,6.483907
5,3,2,7,3301,15766,1300,6.908755,4.320922,4.320922,5.372375,5.159912,5.176513,6.483907
6,4,3,7,3301,15766,1200,11.512936,4.546874,4.546874,5.372375,5.159912,5.176513,3.701200
7,4,2,7,3301,15766,1300,11.512936,4.320922,4.320922,5.372375,5.159912,5.176513,6.483907
8,4,1,7,3301,15766,1300,11.512936,5.537601,5.537601,5.372375,5.159912,5.176513,6.483907
9,4,4,7,3301,15900,1400,11.512936,11.512936,11.512936,5.372375,5.159912,5.099042,6.955416


ClP is not in columns..


ClP
Cliente_ID Producto_ID          
15766      1200         3.701200
           1300         7.758309
15900      1300         1.386294
           1400         6.955416

,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Demanda_uni_equil,lag0par1,A,Ca,R,Cl,P,ClP
0,3,1,7,3301,15766,1200,1.386294,5.537601,5.537601,5.372375,5.159912,5.176513,3.701200,3.701200
1,3,2,7,3301,15766,1200,1.609438,4.320922,4.320922,5.372375,5.159912,5.176513,3.701200,3.701200
2,3,1,7,3301,15766,1200,3.713572,5.537601,5.537601,5.372375,5.159912,5.176513,3.701200,3.701200
3,3,2,7,3301,15766,1300,1.098612,4.320922,4.320922,5.372375,5.159912,5.176513,6.483907,7.758309
4,3,3,7,3301,15900,1300,1.386294,4.546874,4.546874,5.372375,5.159912,5.099042,6.483907,1.386294
5,3,2,7,3301,15766,1300,6.908755,4.320922,4.320922,5.372375,5.159912,5.176513,6.483907,7.758309
6,4,3,7,3301,15766,1200,11.512936,4.546874,4.546874,5.372375,5.159912,5.176513,3.701200,3.701200
7,4,2,7,3301,15766,1300,11.512936,4.320922,4.320922,5.372375,5.159912,5.176513,6.483907,7.758309
8,4,1,7,3301,15766,1300,11.512936,5.537601,5.537601,5.372375,5.159912,5.176513,6.483907,7.758309
9,4,4,7,3301,15900,1400,11.512936,11.512936,11.512936,5.372375,5.159912,5.099042,6.955416,6.955416


,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Demanda_uni_equil,lag0par1,A,Ca,R,Cl,P,ClP
0,3,1,7,3301,15766,1200,1.386294,5.537601,5.537601,5.372375,5.159912,5.176513,3.701200,3.701200
1,3,2,7,3301,15766,1200,1.609438,4.320922,4.320922,5.372375,5.159912,5.176513,3.701200,3.701200
2,3,1,7,3301,15766,1200,3.713572,5.537601,5.537601,5.372375,5.159912,5.176513,3.701200,3.701200
3,3,2,7,3301,15766,1300,1.098612,4.320922,4.320922,5.372375,5.159912,5.176513,6.483907,7.758309
4,3,3,7,3301,15900,1300,1.386294,4.546874,4.546874,5.372375,5.159912,5.099042,6.483907,1.386294
5,3,2,7,3301,15766,1300,6.908755,4.320922,4.320922,5.372375,5.159912,5.176513,6.483907,7.758309
6,4,3,7,3301,15766,1200,11.512936,4.546874,4.546874,5.372375,5.159912,5.176513,3.701200,3.701200
7,4,2,7,3301,15766,1300,11.512936,4.320922,4.320922,5.372375,5.159912,5.176513,6.483907,7.758309
8,4,1,7,3301,15766,1300,11.512936,5.537601,5.537601,5.372375,5.159912,5.176513,6.483907,7.758309
9,4,4,7,3301,15900,1400,11.512936,11.512936,11.512936,5.372375,5.159912,5.099042,6.955416,6.955416


In [14]:
FE.AddConfigurableFeaturesToTrain(configLag2Target1)

lag2A is not in columns..


//anaconda/lib/python2.7/site-packages/pandas/core/indexing.py:415: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


lag2A
Semana Agencia_ID           
5      1            2.549933
       2            3.205602
       3            1.386294
6      1           11.512936
       2           11.512936
       3           11.512936
       4           11.512936

,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Demanda_uni_equil,lag0par1,A,Ca,R,Cl,P,ClP,Lag2Target2,lag2A
0,3,1,7,3301,15766,1200,1.386294,5.537601,5.537601,5.372375,5.159912,5.176513,3.701200,3.701200,NaN,NaN
1,3,2,7,3301,15766,1200,1.609438,4.320922,4.320922,5.372375,5.159912,5.176513,3.701200,3.701200,NaN,NaN
2,3,1,7,3301,15766,1200,3.713572,5.537601,5.537601,5.372375,5.159912,5.176513,3.701200,3.701200,NaN,NaN
3,3,2,7,3301,15766,1300,1.098612,4.320922,4.320922,5.372375,5.159912,5.176513,6.483907,7.758309,NaN,NaN
4,3,3,7,3301,15900,1300,1.386294,4.546874,4.546874,5.372375,5.159912,5.099042,6.483907,1.386294,NaN,NaN
5,3,2,7,3301,15766,1300,6.908755,4.320922,4.320922,5.372375,5.159912,5.176513,6.483907,7.758309,NaN,NaN
6,4,3,7,3301,15766,1200,11.512936,4.546874,4.546874,5.372375,5.159912,5.176513,3.701200,3.701200,NaN,NaN
7,4,2,7,3301,15766,1300,11.512936,4.320922,4.320922,5.372375,5.159912,5.176513,6.483907,7.758309,NaN,NaN
8,4,1,7,3301,15766,1300,11.512936,5.537601,5.537601,5.372375,5.159912,5.176513,6.483907,7.758309,NaN,NaN
9,4,4,7,3301,15900,1400,11.512936,11.512936,11.512936,5.372375,5.159912,5.099042,6.955416,6.955416,NaN,NaN


Count of missing numbers after lag2A in validation part in column Lag2Target2 is 4
lag2Ca is not in columns..


...

Count of missing numbers after lag2R in validation part in column Lag2Target2 is 1
lag2Cl is not in columns..


lag2Cl
Semana Cliente_ID           
5      15766        2.943334
       15900        1.386294
6      15766       11.512936
       15900       11.512936

,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Demanda_uni_equil,lag0par1,A,Ca,R,Cl,P,ClP,Lag2Target2,lag2A,lag2Ca,lag2R,lag2Cl
0,3,1,7,3301,15766,1200,1.386294,5.537601,5.537601,5.372375,5.159912,5.176513,3.701200,3.701200,NaN,NaN,NaN,NaN,NaN
1,3,2,7,3301,15766,1200,1.609438,4.320922,4.320922,5.372375,5.159912,5.176513,3.701200,3.701200,NaN,NaN,NaN,NaN,NaN
2,3,1,7,3301,15766,1200,3.713572,5.537601,5.537601,5.372375,5.159912,5.176513,3.701200,3.701200,NaN,NaN,NaN,NaN,NaN
3,3,2,7,3301,15766,1300,1.098612,4.320922,4.320922,5.372375,5.159912,5.176513,6.483907,7.758309,NaN,NaN,NaN,NaN,NaN
4,3,3,7,3301,15900,1300,1.386294,4.546874,4.546874,5.372375,5.159912,5.099042,6.483907,1.386294,NaN,NaN,NaN,NaN,NaN
5,3,2,7,3301,15766,1300,6.908755,4.320922,4.320922,5.372375,5.159912,5.176513,6.483907,7.758309,NaN,NaN,NaN,NaN,NaN
6,4,3,7,3301,15766,1200,11.512936,4.546874,4.546874,5.372375,5.159912,5.176513,3.701200,3.701200,NaN,NaN,NaN,NaN,NaN
7,4,2,7,3301,15766,1300,11.512936,4.320922,4.320922,5.372375,5.159912,5.176513,6.483907,7.758309,NaN,NaN,NaN,NaN,NaN
8,4,1,7,3301,15766,1300,11.512936,5.537601,5.537601,5.372375,5.159912,5.176513,6.483907,7.758309,NaN,NaN,NaN,NaN,NaN
9,4,4,7,3301,15900,1400,11.512936,11.512936,11.512936,5.372375,5.159912,5.099042,6.955416,6.955416,NaN,NaN,NaN,NaN,NaN


Count of missing numbers after lag2Cl in validation part in column Lag2Target2 is 0


,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Demanda_uni_equil,lag0par1,A,Ca,R,Cl,P,ClP,Lag2Target2,lag2A,lag2Ca,lag2R,lag2Cl
0,3,1,7,3301,15766,1200,1.386294,5.537601,5.537601,5.372375,5.159912,5.176513,3.701200,3.701200,NaN,NaN,NaN,NaN,NaN
1,3,2,7,3301,15766,1200,1.609438,4.320922,4.320922,5.372375,5.159912,5.176513,3.701200,3.701200,NaN,NaN,NaN,NaN,NaN
2,3,1,7,3301,15766,1200,3.713572,5.537601,5.537601,5.372375,5.159912,5.176513,3.701200,3.701200,NaN,NaN,NaN,NaN,NaN
3,3,2,7,3301,15766,1300,1.098612,4.320922,4.320922,5.372375,5.159912,5.176513,6.483907,7.758309,NaN,NaN,NaN,NaN,NaN
4,3,3,7,3301,15900,1300,1.386294,4.546874,4.546874,5.372375,5.159912,5.099042,6.483907,1.386294,NaN,NaN,NaN,NaN,NaN
5,3,2,7,3301,15766,1300,6.908755,4.320922,4.320922,5.372375,5.159912,5.176513,6.483907,7.758309,NaN,NaN,NaN,NaN,NaN
6,4,3,7,3301,15766,1200,11.512936,4.546874,4.546874,5.372375,5.159912,5.176513,3.701200,3.701200,NaN,NaN,NaN,NaN,NaN
7,4,2,7,3301,15766,1300,11.512936,4.320922,4.320922,5.372375,5.159912,5.176513,6.483907,7.758309,NaN,NaN,NaN,NaN,NaN
8,4,1,7,3301,15766,1300,11.512936,5.537601,5.537601,5.372375,5.159912,5.176513,6.483907,7.758309,NaN,NaN,NaN,NaN,NaN
9,4,4,7,3301,15900,1400,11.512936,11.512936,11.512936,5.372375,5.159912,5.099042,6.955416,6.955416,NaN,NaN,NaN,NaN,NaN


In [15]:
FE.AddConfigurableFeaturesToTrain(configLag2Target0)

lag2A is in columns..


,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Demanda_uni_equil,lag0par1,A,Ca,R,Cl,P,ClP,Lag2Target2,lag2A,lag2Ca,lag2R,lag2Cl
0,3,1,7,3301,15766,1200,1.386294,5.537601,5.537601,5.372375,5.159912,5.176513,3.701200,3.701200,NaN,NaN,NaN,NaN,NaN
1,3,2,7,3301,15766,1200,1.609438,4.320922,4.320922,5.372375,5.159912,5.176513,3.701200,3.701200,NaN,NaN,NaN,NaN,NaN
2,3,1,7,3301,15766,1200,3.713572,5.537601,5.537601,5.372375,5.159912,5.176513,3.701200,3.701200,NaN,NaN,NaN,NaN,NaN
3,3,2,7,3301,15766,1300,1.098612,4.320922,4.320922,5.372375,5.159912,5.176513,6.483907,7.758309,NaN,NaN,NaN,NaN,NaN
4,3,3,7,3301,15900,1300,1.386294,4.546874,4.546874,5.372375,5.159912,5.099042,6.483907,1.386294,NaN,NaN,NaN,NaN,NaN
5,3,2,7,3301,15766,1300,6.908755,4.320922,4.320922,5.372375,5.159912,5.176513,6.483907,7.758309,NaN,NaN,NaN,NaN,NaN
6,4,3,7,3301,15766,1200,11.512936,4.546874,4.546874,5.372375,5.159912,5.176513,3.701200,3.701200,NaN,NaN,NaN,NaN,NaN
7,4,2,7,3301,15766,1300,11.512936,4.320922,4.320922,5.372375,5.159912,5.176513,6.483907,7.758309,NaN,NaN,NaN,NaN,NaN
8,4,1,7,3301,15766,1300,11.512936,5.537601,5.537601,5.372375,5.159912,5.176513,6.483907,7.758309,NaN,NaN,NaN,NaN,NaN
9,4,4,7,3301,15900,1400,11.512936,11.512936,11.512936,5.372375,5.159912,5.099042,6.955416,6.955416,NaN,NaN,NaN,NaN,NaN


lag2Ca is in columns..


,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Demanda_uni_equil,lag0par1,A,Ca,R,Cl,P,ClP,Lag2Target2,lag2A,lag2Ca,lag2R,lag2Cl
0,3,1,7,3301,15766,1200,1.386294,5.537601,5.537601,5.372375,5.159912,5.176513,3.701200,3.701200,NaN,NaN,NaN,NaN,NaN
1,3,2,7,3301,15766,1200,1.609438,4.320922,4.320922,5.372375,5.159912,5.176513,3.701200,3.701200,NaN,NaN,NaN,NaN,NaN
2,3,1,7,3301,15766,1200,3.713572,5.537601,5.537601,5.372375,5.159912,5.176513,3.701200,3.701200,NaN,NaN,NaN,NaN,NaN
3,3,2,7,3301,15766,1300,1.098612,4.320922,4.320922,5.372375,5.159912,5.176513,6.483907,7.758309,NaN,NaN,NaN,NaN,NaN
4,3,3,7,3301,15900,1300,1.386294,4.546874,4.546874,5.372375,5.159912,5.099042,6.483907,1.386294,NaN,NaN,NaN,NaN,NaN
5,3,2,7,3301,15766,1300,6.908755,4.320922,4.320922,5.372375,5.159912,5.176513,6.483907,7.758309,NaN,NaN,NaN,NaN,NaN
6,4,3,7,3301,15766,1200,11.512936,4.546874,4.546874,5.372375,5.159912,5.176513,3.701200,3.701200,NaN,NaN,NaN,NaN,NaN
7,4,2,7,3301,15766,1300,11.512936,4.320922,4.320922,5.372375,5.159912,5.176513,6.483907,7.758309,NaN,NaN,NaN,NaN,NaN
8,4,1,7,3301,15766,1300,11.512936,5.537601,5.537601,5.372375,5.159912,5.176513,6.483907,7.758309,NaN,NaN,NaN,NaN,NaN
9,4,4,7,3301,15900,1400,11.512936,11.512936,11.512936,5.372375,5.159912,5.099042,6.955416,6.955416,NaN,NaN,NaN,NaN,NaN


lag2R is in columns..


...

,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Demanda_uni_equil,lag0par1,A,Ca,R,Cl,P,ClP,Lag2Target2,lag2A,lag2Ca,lag2R,lag2Cl,lag2P
0,3,1,7,3301,15766,1200,1.386294,5.537601,5.537601,5.372375,5.159912,5.176513,3.701200,3.701200,NaN,NaN,NaN,NaN,NaN,NaN
1,3,2,7,3301,15766,1200,1.609438,4.320922,4.320922,5.372375,5.159912,5.176513,3.701200,3.701200,NaN,NaN,NaN,NaN,NaN,NaN
2,3,1,7,3301,15766,1200,3.713572,5.537601,5.537601,5.372375,5.159912,5.176513,3.701200,3.701200,NaN,NaN,NaN,NaN,NaN,NaN
3,3,2,7,3301,15766,1300,1.098612,4.320922,4.320922,5.372375,5.159912,5.176513,6.483907,7.758309,NaN,NaN,NaN,NaN,NaN,NaN
4,3,3,7,3301,15900,1300,1.386294,4.546874,4.546874,5.372375,5.159912,5.099042,6.483907,1.386294,NaN,NaN,NaN,NaN,NaN,NaN
5,3,2,7,3301,15766,1300,6.908755,4.320922,4.320922,5.372375,5.159912,5.176513,6.483907,7.758309,NaN,NaN,NaN,NaN,NaN,NaN
6,4,3,7,3301,15766,1200,11.512936,4.546874,4.546874,5.372375,5.159912,5.176513,3.701200,3.701200,NaN,NaN,NaN,NaN,NaN,NaN
7,4,2,7,3301,15766,1300,11.512936,4.320922,4.320922,5.372375,5.159912,5.176513,6.483907,7.758309,NaN,NaN,NaN,NaN,NaN,NaN
8,4,1,7,3301,15766,1300,11.512936,5.537601,5.537601,5.372375,5.159912,5.176513,6.483907,7.758309,NaN,NaN,NaN,NaN,NaN,NaN
9,4,4,7,3301,15900,1400,11.512936,11.512936,11.512936,5.372375,5.159912,5.099042,6.955416,6.955416,NaN,NaN,NaN,NaN,NaN,NaN


lag2S is not in columns..


,lag2S
Semana,
5,2.683828
6,11.512936


,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Demanda_uni_equil,lag0par1,A,Ca,...,Cl,P,ClP,Lag2Target2,lag2A,lag2Ca,lag2R,lag2Cl,lag2P,lag2S
0,3,1,7,3301,15766,1200,1.386294,5.537601,5.537601,5.372375,...,5.176513,3.701200,3.701200,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3,2,7,3301,15766,1200,1.609438,4.320922,4.320922,5.372375,...,5.176513,3.701200,3.701200,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,1,7,3301,15766,1200,3.713572,5.537601,5.537601,5.372375,...,5.176513,3.701200,3.701200,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,2,7,3301,15766,1300,1.098612,4.320922,4.320922,5.372375,...,5.176513,6.483907,7.758309,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3,3,7,3301,15900,1300,1.386294,4.546874,4.546874,5.372375,...,5.099042,6.483907,1.386294,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,3,2,7,3301,15766,1300,6.908755,4.320922,4.320922,5.372375,...,5.176513,6.483907,7.758309,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,4,3,7,3301,15766,1200,11.512936,4.546874,4.546874,5.372375,...,5.176513,3.701200,3.701200,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,4,2,7,3301,15766,1300,11.512936,4.320922,4.320922,5.372375,...,5.176513,6.483907,7.758309,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,4,1,7,3301,15766,1300,11.512936,5.537601,5.537601,5.372375,...,5.176513,6.483907,7.758309,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,4,4,7,3301,15900,1400,11.512936,11.512936,11.512936,5.372375,...,5.099042,6.955416,6.955416,NaN,NaN,NaN,NaN,NaN,NaN,NaN


,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Demanda_uni_equil,lag0par1,A,Ca,...,Cl,P,ClP,Lag2Target2,lag2A,lag2Ca,lag2R,lag2Cl,lag2P,lag2S
0,3,1,7,3301,15766,1200,1.386294,5.537601,5.537601,5.372375,...,5.176513,3.701200,3.701200,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3,2,7,3301,15766,1200,1.609438,4.320922,4.320922,5.372375,...,5.176513,3.701200,3.701200,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,1,7,3301,15766,1200,3.713572,5.537601,5.537601,5.372375,...,5.176513,3.701200,3.701200,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,2,7,3301,15766,1300,1.098612,4.320922,4.320922,5.372375,...,5.176513,6.483907,7.758309,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3,3,7,3301,15900,1300,1.386294,4.546874,4.546874,5.372375,...,5.099042,6.483907,1.386294,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,3,2,7,3301,15766,1300,6.908755,4.320922,4.320922,5.372375,...,5.176513,6.483907,7.758309,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,4,3,7,3301,15766,1200,11.512936,4.546874,4.546874,5.372375,...,5.176513,3.701200,3.701200,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,4,2,7,3301,15766,1300,11.512936,4.320922,4.320922,5.372375,...,5.176513,6.483907,7.758309,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,4,1,7,3301,15766,1300,11.512936,5.537601,5.537601,5.372375,...,5.176513,6.483907,7.758309,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,4,4,7,3301,15900,1400,11.512936,11.512936,11.512936,5.372375,...,5.099042,6.955416,6.955416,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Setup prod

In [11]:
parameterDict = {"ValidationStart":6, 
    "trainHdfPath":'../../input/train.h5',
    "trainHdfFile":"train",
    "testHdfPath":"../../input/test.h5",
    "testHdfFile":"test", 
    "trainTypes" : {'Semana':np.uint8, 'Agencia_ID':np.uint16, 'Canal_ID':np.uint8,'Ruta_SAK':np.uint16, 
        'Cliente_ID':np.uint32, 'Producto_ID':np.uint16,'Demanda_uni_equil':np.uint32}, 
    "testTypes" : {'id':np.uint32,'Semana':np.uint8, 'Agencia_ID':np.uint16, 'Canal_ID':np.uint8,'Ruta_SAK':np.uint16,
        'Cliente_ID':np.uint32, 'Producto_ID':np.uint16},
    "trainCsvPath":'../../input/train.csv'   ,
    "testCsvPath":'../../input/test.csv'}

FE = FeatureEngineering(**parameterDict)
print FE.__dict__

TypeError: __init__() takes exactly 11 arguments (10 given)

## Train and test data, from csv to hdf while log transformation on Demanda..

In [ ]:
#FE.ReadCsv('both')
#FE.Preprocess('train', [["Demanda_uni_equil",np.log1p,'float32']])
#FE.SaveDataFrameToHdf('both')

## Read test and train data from hdf..

In [4]:
FE.ReadHdf('both')

,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Demanda_uni_equil
0,3,1110,7,3301,15766,1212,1.386294
1,3,1110,7,3301,15766,1216,1.609438


Semana                 uint8
Agencia_ID            uint16
Canal_ID               uint8
Ruta_SAK              uint16
Cliente_ID            uint32
Producto_ID           uint16
Demanda_uni_equil    float32
dtype: object
<class 'pandas.core.frame.DataFrame'>
Int64Index: 74180464 entries, 0 to 74180463
Data columns (total 7 columns):
Semana               uint8
Agencia_ID           uint16
Canal_ID             uint8
Ruta_SAK             uint16
Cliente_ID           uint32
Producto_ID          uint16
Demanda_uni_equil    float32
dtypes: float32(1), uint16(3), uint32(1), uint8(2)
memory usage: 1.4 GB
None


,id,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID
0,0,11,4037,1,2209,4639078,35305
1,1,11,2237,1,1226,4705135,1238


id             uint32
Semana          uint8
Agencia_ID     uint16
Canal_ID        uint8
Ruta_SAK       uint16
Cliente_ID     uint32
Producto_ID    uint16
dtype: object
<class 'pandas.core.frame.DataFrame'>
Int64Index: 6999251 entries, 0 to 6999250
Data columns (total 7 columns):
id             uint32
Semana         uint8
Agencia_ID     uint16
Canal_ID       uint8
Ruta_SAK       uint16
Cliente_ID     uint32
Producto_ID    uint16
dtypes: uint16(3), uint32(2), uint8(2)
memory usage: 133.5 MB
None


## Test setup..

In [9]:
parameterDict = {"ValidationStart":6, 
    "ValidationSmallerThan":7,
    "maxLag":2,    
    "trainHdfPath":'../../input/train_100.h5',
    "trainHdfFile":"train",
    "testHdfPath":"../../input/test.h5",
    "testHdfFile":"test", 
    "trainTypes" : {'Semana':np.uint8, 'Agencia_ID':np.uint16, 'Canal_ID':np.uint8,'Ruta_SAK':np.uint16, 
        'Cliente_ID':np.uint32, 'Producto_ID':np.uint16,'Demanda_uni_equil':np.uint32}, 
    "testTypes" : {'id':np.uint32,'Semana':np.uint8, 'Agencia_ID':np.uint16, 'Canal_ID':np.uint8,'Ruta_SAK':np.uint16,
        'Cliente_ID':np.uint32, 'Producto_ID':np.uint16},
    "trainCsvPath":'../../input/train_100.csv'   ,
    "testCsvPath":'../../input/test.csv'}

FE = FeatureEngineering(**parameterDict)
print FE.__dict__

{'maxLag': 2, 'testTypes': {'Cliente_ID': <type 'numpy.uint32'>, 'Ruta_SAK': <type 'numpy.uint16'>, 'Canal_ID': <type 'numpy.uint8'>, 'Producto_ID': <type 'numpy.uint16'>, 'Agencia_ID': <type 'numpy.uint16'>, 'Semana': <type 'numpy.uint8'>, 'id': <type 'numpy.uint32'>}, 'testHdfFile': 'test', 'trainTypes': {'Cliente_ID': <type 'numpy.uint32'>, 'Demanda_uni_equil': <type 'numpy.uint32'>, 'Ruta_SAK': <type 'numpy.uint16'>, 'Canal_ID': <type 'numpy.uint8'>, 'Producto_ID': <type 'numpy.uint16'>, 'Agencia_ID': <type 'numpy.uint16'>, 'Semana': <type 'numpy.uint8'>}, 'trainCsvPath': '../../input/train_100.csv', 'testHdfPath': '../../input/test.h5', 'testCsvPath': '../../input/test.csv', 'ValidationStart': 6, 'trainHdfFile': 'train', 'trainHdfPath': '../../input/train_100.h5'}


In [10]:
#FE.ReadCsv('train')
#FE.Preprocess('train', [["Demanda_uni_equil",np.log1p,'float32']])
#FE.SaveDataFrameToHdf('train')

,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Demanda_uni_equil
0,3,1,7,3301,15766,1200,3
1,3,2,7,3301,15766,1200,4


Semana                uint8
Agencia_ID           uint16
Canal_ID              uint8
Ruta_SAK             uint16
Cliente_ID           uint32
Producto_ID          uint16
Demanda_uni_equil    uint32
dtype: object
<class 'pandas.core.frame.DataFrame'>
Int64Index: 19 entries, 0 to 18
Data columns (total 7 columns):
Semana               19 non-null uint8
Agencia_ID           19 non-null uint16
Canal_ID             19 non-null uint8
Ruta_SAK             19 non-null uint16
Cliente_ID           19 non-null uint32
Producto_ID          19 non-null uint16
Demanda_uni_equil    19 non-null uint32
dtypes: uint16(3), uint32(2), uint8(2)
memory usage: 456.0 bytes
None


In [11]:
FE.ReadHdf('train')

,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Demanda_uni_equil
0,3,1,7,3301,15766,1200,1.386294
1,3,2,7,3301,15766,1200,1.609438


Semana                 uint8
Agencia_ID            uint16
Canal_ID               uint8
Ruta_SAK              uint16
Cliente_ID            uint32
Producto_ID           uint16
Demanda_uni_equil    float32
dtype: object
<class 'pandas.core.frame.DataFrame'>
Int64Index: 19 entries, 0 to 18
Data columns (total 7 columns):
Semana               19 non-null uint8
Agencia_ID           19 non-null uint16
Canal_ID             19 non-null uint8
Ruta_SAK             19 non-null uint16
Cliente_ID           19 non-null uint32
Producto_ID          19 non-null uint16
Demanda_uni_equil    19 non-null float32
dtypes: float32(1), uint16(3), uint32(1), uint8(2)
memory usage: 380.0 bytes
None


In [14]:
display(FE.AddDemandaGeneralMean())

,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Demanda_uni_equil,DemandaGeneralMean
0,3,1110,7,3301,15766,1200,1.386294,5.159912
1,3,1110,7,3301,15766,1200,1.609438,5.159912
2,3,1110,7,3301,15766,1200,3.713572,5.159912
3,3,1110,7,3301,15766,1300,1.098612,5.159912
4,3,1110,7,3301,15900,1300,1.386294,5.159912
5,3,1110,7,3301,15766,1300,6.908755,5.159912
6,4,1110,7,3301,15766,1200,11.512936,5.159912
7,4,1110,7,3301,15766,1300,11.512936,5.159912
8,4,1110,7,3301,15766,1300,11.512936,5.159912
9,4,1110,7,3301,15900,1400,11.512936,5.159912


None

In [28]:
qw = FE.train.loc[FE.train.loc[:,'Semana']>4]
qw


,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Demanda_uni_equil,DemandaGeneralMean,lag2A,lag2Ca,lag2R,lag2Cl,lag2P,lag2S
10,5,1110,7,3301,15766,1200,2.397895,5.159912,2.683828,2.683828,2.683828,2.943334,2.236435,2.683828
11,5,1110,7,3301,15766,1200,2.397895,5.159912,2.683828,2.683828,2.683828,2.943334,2.236435,2.683828
12,5,1110,7,3301,15766,1200,2.890372,5.159912,2.683828,2.683828,2.683828,2.943334,2.236435,2.683828
13,5,1110,7,3301,15900,1400,2.397895,5.159912,2.683828,2.683828,2.683828,1.386294,NaN,2.683828
14,6,1110,7,3301,15766,1400,5.298317,5.159912,11.512936,11.512936,11.512936,11.512936,11.512936,11.512936


In [79]:


FE.train.loc[:,'target'] = np.nan
FE.train

,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Demanda_uni_equil,DemandaGeneralMean,lag2A,lag2Ca,lag2R,lag2Cl,lag2P,lag2S,target
0,3,1110,7,3301,15766,1200,1.386294,5.159912,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3,1110,7,3301,15766,1200,1.609438,5.159912,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,1110,7,3301,15766,1200,3.713572,5.159912,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,1110,7,3301,15766,1300,1.098612,5.159912,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3,1110,7,3301,15900,1300,1.386294,5.159912,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,3,1110,7,3301,15766,1300,6.908755,5.159912,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,4,1110,7,3301,15766,1200,11.512936,5.159912,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,4,1110,7,3301,15766,1300,11.512936,5.159912,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,4,1110,7,3301,15766,1300,11.512936,5.159912,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,4,1110,7,3301,15900,1400,11.512936,5.159912,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [82]:
FE.train.loc[pd.isnull(FE.train['target']), 'target'] = FE.train.loc[pd.isnull(FE.train['target']), 'lag2P']
FE.train["target"]

0           NaN
1           NaN
2           NaN
3           NaN
4           NaN
5           NaN
6           NaN
7           NaN
8           NaN
9           NaN
10     2.236435
11     2.236435
12     2.236435
13          NaN
14    11.512936
Name: target, dtype: float32

In [87]:
FE.train.loc[FE.train['Semana']>4,"target"].isnull().sum()

10     2.236435
11     2.236435
12     2.236435
13          NaN
14    11.512936
Name: target, dtype: float32

In [88]:
FE.train.loc[FE.train['Semana']>4,"target"].isnull().sum()

1

In [89]:
FE.train.loc[pd.isnull(FE.train['target']), 'target'] = FE.train.loc[pd.isnull(FE.train['target']), 'lag2Cl']
FE.train["target"]

0           NaN
1           NaN
2           NaN
3           NaN
4           NaN
5           NaN
6           NaN
7           NaN
8           NaN
9           NaN
10     2.236435
11     2.236435
12     2.236435
13     1.386294
14    11.512936
Name: target, dtype: float32

In [90]:
FE.train.loc[FE.train['Semana']>4,"target"].isnull().sum()

0

In [45]:
"Semana" not in FE.train.columns

False

In [46]:
FE.train

,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Demanda_uni_equil,lag0par1,lag2par1
0,3,1,7,3301,15766,1200,1.386294,5.537601,NaN
1,3,2,7,3301,15766,1200,1.609438,4.320922,NaN
2,3,1,7,3301,15766,1200,3.713572,5.537601,NaN
3,3,2,7,3301,15766,1300,1.098612,4.320922,NaN
4,3,3,7,3301,15900,1300,1.386294,4.546874,NaN
5,3,2,7,3301,15766,1300,6.908755,4.320922,NaN
6,4,3,7,3301,15766,1200,11.512936,4.546874,NaN
7,4,2,7,3301,15766,1300,11.512936,4.320922,NaN
8,4,1,7,3301,15766,1300,11.512936,5.537601,NaN
9,4,4,7,3301,15900,1400,11.512936,11.512936,NaN


In [64]:
print "{} haha {}".format(1,2)

1 haha 2
